# Imports

In [1]:
from pydantic import BaseModel, Field


from langsmith import traceable


from langchain_core.messages import AIMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Dict, Any, Annotated, List
from operator import add


import instructor

from litellm import completion

/Users/eliekawerk/PycharmProjects/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create a Coordinator Agent with LiteLLM Router and Feedback model

In [2]:
class Delegation(BaseModel):
    agent: str
    task: str


class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: List[Delegation]
    final_answer: bool
    answer: str

In [ ]:
@traceable(
    name="coordinator_agent",
    run_type="llm",
    metadata={"ls_provider": "openai", "ls_model_name": "gpt-4.1"},
)
def coordinator_agent(state, models=["gpt-4.1-mini", "groq/llama-3.3-70b-versatile"]):
    prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.
- warehouse_manager_agent: The user is asking to reserve items from the warehouses or about availability of the items in warehouses.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true
- One of the following has to be true:
next_agent is "" and final_answer is true
next_agent is not "" and final_answer is false

Additional instructions:

- Do not route to any agent if the user's query needs clarification. Do it yourself.
- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

    template = Template(prompt_template)

    prompt = template.render()

    messages = state.messages

    conversation = []

    for message in messages:
        conversation.append(convert_to_openai_messages(message))

    client = instructor.from_litellm(completion)

    for model in models:
        try:
            response, raw_response = client.chat.completions.create_with_completion(
                model=model,
                response_model=CoordinatorAgentResponse,
                messages=[{"role": "system", "content": prompt}, *conversation],
                temperature=0.0,
            )
            break
        except Exception as e:
            print(f"Error with model {model}: {e}")
            continue

    if response.final_answer:
        ai_message = []
    else:
        ai_message = [AIMessage(content=response.answer)]

    return {
        "messages": ai_message,
        "answer": response.answer,
        "coordinator_agent": {
            "iteration": state.coordinator_agent.iteration + 1,
            "final_answer": response.final_answer,
            "next_agent": response.next_agent,
            "plan": [data.model_dump() for data in response.plan],
        },
    }

In [4]:
class ToolCall(BaseModel):
    name: str
    arguments: dict


class RAGUsedContext(BaseModel):
    id: str = Field(description="ID of the item used to answer the question.")
    description: str = Field(
        description="Short description of the item used to answer the question."
    )


class AgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []


class CoordinatorAgentProperties(BaseModel):
    iteration: int = 0
    final_answer: bool = False
    plan: List[Delegation] = []
    next_agent: str = ""


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    user_intent: str = ""
    product_qa_agent: AgentProperties = Field(default_factory=AgentProperties)
    shopping_cart_agent: AgentProperties = Field(default_factory=AgentProperties)
    coordinator_agent: CoordinatorAgentProperties = Field(
        default_factory=AgentProperties
    )
    answer: str = ""
    references: Annotated[List[RAGUsedContext], add] = []
    user_id: str = ""
    cart_id: str = ""

In [ ]:
initial_state = State(
    messages=[{"role": "user", "content": "What is the weather today?"}]
)

In [ ]:
answer = coordinator_agent(
    initial_state, models=["gpt-4.1-mini", "groq/llama-3.3-70b-versatile"]
)

In [7]:
answer

{'messages': [AIMessage(content='I cannot provide weather information. Please ask about products, shopping, or warehouse-related queries.', additional_kwargs={}, response_metadata={})],
 'answer': 'I cannot provide weather information. Please ask about products, shopping, or warehouse-related queries.',
 'coordinator_agent': {'iteration': 1,
  'final_answer': False,
  'next_agent': '',
  'plan': []}}

In [8]:
answer = coordinator_agent(
    initial_state, models=["groq/llama-3.3-70b-versatile", "gpt-4.1-mini"]
)

In [9]:
answer

{'messages': [AIMessage(content='I need to clarify your query. The weather can vary depending on the location. Could you please provide me with the city or region you are interested in?', additional_kwargs={}, response_metadata={})],
 'answer': 'I need to clarify your query. The weather can vary depending on the location. Could you please provide me with the city or region you are interested in?',
 'coordinator_agent': {'iteration': 1,
  'final_answer': False,
  'next_agent': '',
  'plan': [{'agent': 'CoordinatorAgent', 'task': 'Clarify user query'}]}}